In [1]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pinecone (from versions: none)
ERROR: No matching distribution found for pinecone


In [ ]:
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate
import os
import sys
%pip install pinecone-client
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain

In [18]:
import os
from getpass import getpass

if not os.environ.get("GRADIENT_ACCESS_TOKEN", None):
    os.environ["GRADIENT_ACCESS_TOKEN"] = getpass("gradient.ai access token:")
if not os.environ.get("GRADIENT_WORKSPACE_ID", None):
    os.environ["GRADIENT_WORKSPACE_ID"] = getpass("gradient.ai workspace id:")

In [53]:
import pinecone
pinecone.init(api_key='381d92f7-5aa9-479a-b691-3706db457cb7', environment='gcp-starter')

In [54]:
import gradientai

client = gradientai.Gradient()

models = client.list_models(only_base=True)
for model in models:
    print(model.id)

99148c6d-c2a0-4fbe-a4a7-e7c05bdb8a09_base_ml_model
f0b97d96-51a8-4040-8b22-7940ee1fa24e_base_ml_model
cc2dafce-9e6e-4a23-a918-cad6ba89e42e_base_ml_model


In [55]:
new_model = models[-1].create_model_adapter(name="my_model_adapter")
new_model.id, new_model.name

('d0afdffd-618e-4e30-a32b-18aabd2f962e_model_adapter', 'my_model_adapter')

In [56]:
llm = GradientLLM(
    model="d0afdffd-618e-4e30-a32b-18aabd2f962e_model_adapter",
    model_kwargs=dict(max_generated_token_count=128),
)

In [65]:

loader = PyPDFLoader('./132-574-1-PB.pdf')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [66]:
embeddings = HuggingFaceEmbeddings()

In [67]:
index_name = "llama2"
index = pinecone.Index(index_name)
vectordb = Pinecone.from_documents(texts, embeddings, index_name=index_name)
print(texts)

[Document(page_content='International Journal of STEM Education for Sustainability , Vol.3, No. 1, 2023, pp. 47-67 \ne-ISSN 2798 -5091 . DOI. 10.52889/ ijses.v3i1.132 \n47 \n Addressing Cybersecurity Challenges in Education  \nSubmitted 29 June 2022 , Revised 20 August 2022 , Accepted 20 August 2022  \n \nWilliam J. Triplett1  \n \n1Cybersecurity Leadership  Program, Capitol Technology University , Laurel , United States \nCoresponding Email: *wjtriplett@captechu.edu : \n \nAbstract  \n \nThis study aim ed to address the challenges of cybersecurity in education. As kindergarten through twelfth -grade \neducation shifts to online and remote learning, educators and governments are increasingly vulnerable to the risks of \ncyberattacks and cybercrimes. This study focused on the strategies that institutions can employ  to increase their \nstudents’ cybersec urity awareness and simultaneously motivate them to pursue cybersecurity as a career . To achieve \nthe research objectives, a systema

In [72]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True
)

In [74]:
chat_history = []
while True:
    query = input('Prompt: ')
    if query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('question:' + query + '\n' 'Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))
    


question:what is the title of the paper?
Answer:  The title of the paper is "International Journal of STEM Education for Sustainability, Vol.3, No. 1, 2023, pp. 47-67 e-ISSN 2798 -5091 . DOI. 10.52889/ ijses.v3i1.132"

question:what is summary of the paper?
Answer:  The paper discusses the importance of STEM education for sustainability and presents a systematic review of studies that use digital games to increase cybersecurity awareness among high school and university students. The paper highlights the need for more research in this area and provides recommendations for future studies.

question:
Answer:  STEM education for sustainability is important because it promotes the development of critical thinking, problem-solving, and creativity skills that are necessary for addressing complex environmental, social, and economic challenges. It also fosters an interdisciplinary approach to learning, which is essential for understanding the interconnectedness of sustainability issues. Additi

SystemExit: 

c:\Users\tanis\miniconda3\envs\Desktop\lib\site-packages\IPython\core\interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
